In [ ]:
import requests
from collections import Counter
from web3 import Web3, HTTPProvider
connection = Web3(HTTPProvider('https://mainnet.infura.io/v3/6873c06628bc4c50af981c610f2f0fc1'))
current_block = connection.eth.blockNumber
print ("Latest Ethereum block number", current_block)

In [ ]:
number_of_blocks = 100 #ca 5400 blocks per day so 5400 * 30 162'000 blocks per month
starting_block = current_block - number_of_blocks

candidates_dict = {}

for i in range(number_of_blocks):
    # print(i, " of ", number_of_blocks)
    block_number = starting_block - i
    block = connection.eth.get_block(block_number)
    hex_transaction_hashes = block["transactions"]

    for i in hex_transaction_hashes:
        transaction_hash = i.hex() 
        transaction = connection.eth.get_transaction(transaction_hash)
        value = transaction["value"] / (10**18)
        if (value > 0.2):
            receiver = transaction["to"] 
            if (receiver in candidates_dict):
                list = candidates_dict[receiver]
                list.append(value)
                candidates_dict[receiver] = list
            else:
                candidates_dict[receiver] = [value]
            

# candidates_dict

In [ ]:
all_info_for_addresses = []
transactions_from_most_common_sender_for_addresses = []

most_common_sender_per_address = []

counter = len(candidates_dict)
updates = 0

for address in candidates_dict.keys():
    print(updates, " of ", counter)
    updates += 1
    try:
        url = f"https://api.covalenthq.com/v1/1/address/{address}/transactions_v2/?quote-currency=USD&page-size=150&key=ckey_06de8b133f2948eb88f41f10539"
        res = requests.get(url).json()

        transactions = res["data"]["items"]
        transactions_cleaned = []
        sender_addresses = []

        num_transaction_in_22 = len([1 for x in transactions if x["block_signed_at"][0:4] == "2022"])
        if (num_transaction_in_22 > 100):
            #print("skipping")
            continue

        for trans in transactions:
            if(trans["value_quote"] is not None and trans["value_quote"] > 100 and trans["from_address"] != address):
                sender_addresses.append(trans["from_address"])
                transactions_cleaned.append({"timestamp": trans["block_signed_at"], "from": trans["from_address"], "from_label": trans["from_address_label"], "value": float(trans["value"]) / (10**18), "valueUSD": trans["value_quote"]})

        all_info_for_addresses.append({"address": address, "data": transactions_cleaned})

        c = Counter(sender_addresses)
        most_common_address = c.most_common(1)[0][0]

        transactions_from_most_common_sender = []
        year_months = []
        for trans in transactions_cleaned:
            if (trans["from"] == most_common_address):
                transactions_from_most_common_sender.append(trans)
                year_months.append(trans["timestamp"][0:7])

        if (len(set(year_months)) >= 10):
            transactions_from_most_common_sender_for_addresses.append({"address": address, "data": transactions_from_most_common_sender})
            most_common_sender_per_address.append({"address": address, "payer_data": c.most_common(1)})

        #print(transactions_from_most_common_sender)
        #print(most_common_sender_per_address)

        if (counter == 0):
            break
        else:
            print(counter)
    except:
        print("error came up - continuing")
        continue

In [ ]:
most_common_sender_per_address